In [1]:
from langchain_core.documents import Document as LangchainDocument
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_community.vectorstores import FAISS
from langchain_huggingface import HuggingFaceEmbeddings
import pandas as pd

dados = pd.read_csv('data/processed/v1_processed_articles.csv')
dados.head()


/home/julio/Documentos/agents_pos/amldo/AMLDO/venv2/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


,lei,titulo,capitulo,artigo,path,texto
0,D10024,TITULO_0,CAPITULO_0,artigo_0.txt,D10024/TITULO_0/capitulos/CAPITULO_0/artigos/a...,Presidência da República\nSecretaria-Geral\nSu...
1,D10024,TITULO_0,CAPITULO_I,artigo_0.txt,D10024/TITULO_0/capitulos/CAPITULO_I/artigos/a...,DISPOSIÇÕES PRELIMINARES\nObjeto e âmbito de a...
2,D10024,TITULO_0,CAPITULO_I,artigo_1.txt,D10024/TITULO_0/capitulos/CAPITULO_I/artigos/a...,"Art. 1º Este Decreto regulamenta a licitação, ..."
3,D10024,TITULO_0,CAPITULO_I,artigo_2.txt,D10024/TITULO_0/capitulos/CAPITULO_I/artigos/a...,"Art. 2º O pregão, na forma eletrônica, é condi..."
4,D10024,TITULO_0,CAPITULO_I,artigo_3.txt,D10024/TITULO_0/capitulos/CAPITULO_I/artigos/a...,"Art. 3º Para fins do disposto neste Decreto, c..."


In [2]:
dados.columns

Index(['lei', 'titulo', 'capitulo', 'artigo', 'path', 'texto'], dtype='object')

# Base bruta

In [3]:
BASE_DADOS_BRUTA = [
    LangchainDocument(
        page_content=row["texto"],
        metadata={
            "lei": row['lei'], 
            "titulo": row["titulo"], 
            "capitulo": row["capitulo"], 
            "artigo": row["artigo"], 
            # "tokens": row["tokens"], 
            "path": row["path"]
        }
    )
    for _, row in dados.iterrows()
]

BASE_DADOS_BRUTA[11:12]

[Document(metadata={'lei': 'D10024', 'titulo': 'TITULO_0', 'capitulo': 'CAPITULO_III', 'artigo': 'artigo_0.txt', 'path': 'D10024/TITULO_0/capitulos/CAPITULO_III/artigos/artigo_0.txt'}, page_content='DO ACESSO AO SISTEMA ELETRÔNICO\nCredenciamento')]

# Processamento de documentos

In [4]:
separador_texto = RecursiveCharacterTextSplitter(
    chunk_size=1000,
    chunk_overlap=150,
    add_start_index=True,
    strip_whitespace=True,
)

documentos_processados = []
for doc_id, doc in enumerate(BASE_DADOS_BRUTA):
    chunks = separador_texto.split_documents([doc])
    for i, ch in enumerate(chunks):
        base_meta = doc.metadata.copy()
        ch.metadata = {
            **base_meta,
            **ch.metadata,     # preserva start_index, se veio
            "chunk_idx": i,
            "orig_doc_id": doc_id,
        }
        documentos_processados.append(ch)

print(f"Total de chunks: {len(documentos_processados)}")

Total de chunks: 1139


# Get vectors db

In [5]:
# CPU (sem torch)
modelo_embedding = HuggingFaceEmbeddings(
    model_name="sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2",
    encode_kwargs={"normalize_embeddings": True},
)

vector_db = FAISS.from_documents(documentos_processados, modelo_embedding)
vector_db.save_local("data/vector_db/v1_faiss_vector_db")

# Simple sample query

In [6]:
pergunta = "dispensa de licitação por valor"
resultado = vector_db.similarity_search_with_score(pergunta, k=8)

for i, (doc, score) in enumerate(resultado, start=1):
    md = doc.metadata or {}
    print(f"Documento {i} | score={score:.4f}")
    print(f"Lei: {md.get('lei')} | Título: {md.get('titulo')} | Capítulo: {md.get('capitulo')} | Artigo: {md.get('artigo')} | Chunk: {md.get('chunk_idx')}")
    print(f"Path: {md.get('path')}")
    print(doc.page_content[:500], "\n", "--"*60, "\n")


Documento 1 | score=0.5326
Lei: D10024 | Título: TITULO_0 | Capítulo: CAPITULO_IV | Artigo: artigo_0.txt | Chunk: 0
Path: D10024/TITULO_0/capitulos/CAPITULO_IV/artigos/artigo_0.txt
DA CONDUÇÃO DO PROCESSO
Órgão ou entidade promotora da licitação 
 ------------------------------------------------------------------------------------------------------------------------ 

Documento 2 | score=0.6599
Lei: L14133 | Título: TITULO_II | Capítulo: CAPITULO_VIII | Artigo: artigo_75.txt | Chunk: 0
Path: L14133/TITULO_II/capitulos/CAPITULO_VIII/artigos/artigo_75.txt
Art. 75. É dispensável a licitação:
I - para contratação que envolva valores inferiores a R$ 100.000,00 (cem mil reais), no caso de obras e serviços
de engenharia ou de serviços de manutenção de veículos automotores; (Vide Decreto nº 10.922, de 2021)
(Vigência) (Vide Decreto nº 11.317, de 2022) Vigência (Vide Decreto nº 11.871, de 2023) Vigência (Vide
Decreto nº 12.343, de 2024) Vigência
II - para contratação que envolva valores inferio

# Load Test

In [7]:
# 1) Embeddings (iguais aos usados na hora de salvar!)
modelo_embedding = HuggingFaceEmbeddings(
    model_name="sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2",
    encode_kwargs={"normalize_embeddings": True},
)

# 2) Carregar o índice salvo
vector_db = FAISS.load_local(
    "data/vector_db/v1_faiss_vector_db",
    embeddings=modelo_embedding,
    allow_dangerous_deserialization=True,   # necessário nas versões novas do LC
)

# 3) Sua busca (sem mudar nada)
pergunta = "dispensa de licitação por valor"
resultado = vector_db.similarity_search_with_score(pergunta, k=8)

for i, (doc, score) in enumerate(resultado, start=1):
    md = doc.metadata or {}
    print(f"Documento {i} | score={score:.4f}")
    print(f"Lei: {md.get('lei')} | Título: {md.get('titulo')} | Capítulo: {md.get('capitulo')} | Artigo: {md.get('artigo')} | Chunk: {md.get('chunk_idx')}")
    print(f"Path: {md.get('path')}")
    print(doc.page_content[:500], "\n", "--"*60, "\n")

Documento 1 | score=0.5326
Lei: D10024 | Título: TITULO_0 | Capítulo: CAPITULO_IV | Artigo: artigo_0.txt | Chunk: 0
Path: D10024/TITULO_0/capitulos/CAPITULO_IV/artigos/artigo_0.txt
DA CONDUÇÃO DO PROCESSO
Órgão ou entidade promotora da licitação 
 ------------------------------------------------------------------------------------------------------------------------ 

Documento 2 | score=0.6599
Lei: L14133 | Título: TITULO_II | Capítulo: CAPITULO_VIII | Artigo: artigo_75.txt | Chunk: 0
Path: L14133/TITULO_II/capitulos/CAPITULO_VIII/artigos/artigo_75.txt
Art. 75. É dispensável a licitação:
I - para contratação que envolva valores inferiores a R$ 100.000,00 (cem mil reais), no caso de obras e serviços
de engenharia ou de serviços de manutenção de veículos automotores; (Vide Decreto nº 10.922, de 2021)
(Vigência) (Vide Decreto nº 11.317, de 2022) Vigência (Vide Decreto nº 11.871, de 2023) Vigência (Vide
Decreto nº 12.343, de 2024) Vigência
II - para contratação que envolva valores inferio

In [8]:
import os
from langchain.chat_models import init_chat_model
from dotenv import load_dotenv
load_dotenv()

# os.environ["GOOGLE_API_KEY"] = os.environ["GOOGLE_API_KEY"]
llm = init_chat_model("gemini-2.5-flash", model_provider="google_genai")

In [9]:
# imports que existem no langchain moderno (core)
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnablePassthrough
from langchain_core.output_parsers import StrOutputParser

# seu retriever continua igual
retriever = vector_db.as_retriever(search_type="mmr", search_kwargs={"k": 5})

prompt = ChatPromptTemplate.from_template(
    "Use APENAS o contexto para responder.\n\nContexto:\n{context}\n\nPergunta:\n{question}"
)

# pipeline RAG sem langchain.chains
rag_chain = (
    {"context": retriever, "question": RunnablePassthrough()}
    | prompt
    | llm
    | StrOutputParser()
)

pergunta = "dispensa de licitação por valor, o que é?"
resposta = rag_chain.invoke(pergunta)

print("Pergunta:", pergunta)
print("Resposta:", resposta)


Pergunta: dispensa de licitação por valor, o que é?
Resposta: De acordo com o contexto fornecido, a dispensa de licitação por valor é aplicável nos seguintes casos:

*   Para contratação que envolva valores inferiores a R$ 100.000,00 (cem mil reais), no caso de obras e serviços de engenharia ou de serviços de manutenção de veículos automotores.
*   Para contratação que envolva valores inferiores a R$ 50.000,00 (cinquenta mil reais), no caso de outros serviços e compras.
